In [5]:
import os
import ray
import requests
import numpy as np
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

import sys
sys.path.append("../scripts/04_extract_frame_features/")
from frame_feature_extractor import FrameFeatureExtractor

from typing import List

class BLIP2VQAFrameFeatureExtractor(FrameFeatureExtractor):
    def __init__(self):
        super().__init__()
        self.processor = Blip2Processor.from_pretrained(
            os.path.join(os.environ["SCRATCH"], "mq_libs/blip2")
        )
        self.model = Blip2ForConditionalGeneration.from_pretrained(
            os.path.join(os.environ["SCRATCH"], "mq_libs/blip2"),
            torch_dtype=torch.float16,
        )
        self.model.to("cuda")
        self.questions = [
            "What does the image describe?",
            "What is the person in this picture doing?",
            "What is happening in this picture?",
        ]

    @property
    def column_names(self):
        return ["frame_index", "question", "answer"]

    def predictor_function(
        self, frame_indices_batch: List[int], frames_batch: List[np.array]
    ):
        # with torch.no_grad():
        #     all_generated_texts = []
            # for question in self.questions:
        url = "http://images.cocodataset.org/val2017/000000039769.jpg"
        image = Image.open(requests.get(url, stream=True).raw)

        prompt = "Question: how many cats are there? Answer:"
        inputs = self.processor(images=image, text=prompt, return_tensors="pt").to("cuda", torch.float16)

        generated_ids = self.model.generate(**inputs)
        generated_text = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        print(generated_text)


In [6]:
import cv2

cap = cv2.VideoCapture(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/clips/00182baf-e3fe-4bee-9416-825555bc4506.mp4"))


In [7]:
processor = Blip2Processor.from_pretrained(
    os.path.join(os.environ["SCRATCH"], "mq_libs/blip2")
)
model = Blip2ForConditionalGeneration.from_pretrained(
    os.path.join(os.environ["SCRATCH"], "mq_libs/blip2"),
    torch_dtype=torch.float16,
)
model.to("cuda")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/data/aarslan/mambaforge/envs/mq_data/lib/python3.9/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [12]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

prompt = "Question: How many cats are there? Answer:"
inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda", torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()


In [13]:
generated_text


'Two'